In [34]:
import nltk
import pandas as pd
from nltk.tokenize import TweetTokenizer
import random
# http://www.nltk.org/book/ch06.html
# http://www.nltk.org/api/nltk.tokenize.html
# http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/
# http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/

In [35]:
tweets_df = pd.read_csv('Sentiment-Analysis-Dataset.csv', error_bad_lines=False)
print(tweets_df.shape[0])


b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


1578612


In [36]:
# store positive and negative tweets
tweet_negative = []
tweet_positive = []

# separate the negative and positive tweets
tweet_negative = tweets_df.loc[tweets_df['Sentiment'] == 0]
tweet_positive = tweets_df.loc[tweets_df['Sentiment'] == 1]


tweet_negative['Sentiment'].replace(0, 'negative',inplace=True)
tweet_positive['Sentiment'].replace(1, 'positive',inplace=True)

# only keep tweets and their label as a list
df_negative = list(zip(tweet_negative['SentimentText'], tweet_negative['Sentiment']))
df_positive = list(zip(tweet_positive['SentimentText'], tweet_positive['Sentiment']))
        
        
        

/home/nicolas/.local/lib/python3.5/site-packages/pandas/core/generic.py:3813: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [37]:
# create a single list of tuples each containing two elements
# First element is an array containing the words and the second element is the type of sentimenet
# We get rid of the words smaller than 2 characaters and we use lowercase for everything

# tokenize the tweets
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

tweets = []
for (words, sentiment) in df_negative + df_positive:
    words = tknzr.tokenize(words)
    words_filtered = []
    for e in words:
        e = e.lower()
        if ':' in e:
            e = e.replace(':', '')
        if '#' in e:
            e = e.replace('#', '')
        if '!' in e:
            e = e.replace('!', '')
        if '?' in e:
            e = e.replace('?', '')
        if ';' in e:
            e = e.replace(';', '')
        if '.' in e:
            e = e.replace('.', '')
        if ',' in e:
            e = e.replace(',', '')
        if '@' in e:
            e = e.replace('@', '')
        if len(e) >= 3:
            words_filtered.append(e)
    tweets.append((words_filtered, sentiment))
        
        
    #words_filtered = [e.lower() for e in words.split() if len(e) >= 3 and ':' not in e and '@' not in e and '!' not in e  and not '?' in e and not '.' in e and not ';' in e and not ',']
    #tweets.append((words_filtered, sentiment))
    
# shuffle the tweets dataset
random.shuffle(tweets)


In [38]:
a = '@remy: This is #waaaaayyyy too much for you!!!!!!'
a = tknzr.tokenize(a)
print(a)

[':', 'This', 'is', '#waaayyy', 'too', 'much', 'for', 'you', '!', '!', '!']


In [39]:
# training and test set
# we keep 90% for the training set and 10% for the test set
limit_set = int(tweets_df.shape[0]/10)
test_set = tweets[:1000]
training_set = tweets[1000:10000]


In [40]:
test_set

[(['england',
   'when',
   'older',
   'want',
   'cornwall',
   'have',
   'you',
   'ever',
   'been',
   'there',
   "there's",
   'beatiful',
   'and',
   'london'],
  'positive'),
 (['boreeed'], 'negative'),
 (['just', 'totaly', 'made', 'day'], 'positive'),
 (['restless',
   'and',
   'bored',
   'seems',
   'like',
   "everyone's",
   'got',
   'something',
   'going',
   'tonight',
   'except'],
  'negative'),
 (['thanks',
   'for',
   'the',
   'dark',
   'hunter',
   'info',
   'all',
   'awesome',
   'everyone',
   'said',
   'order'],
  'positive'),
 (['well',
   'the',
   'menu',
   'today',
   'little',
   'this',
   'http//owly/ef2p',
   'and',
   'little',
   'that',
   'http//owly/ef3z',
   'what',
   'you',
   'think'],
  'positive'),
 (['job',
   'going',
   'well',
   'looking',
   'forward',
   'the',
   'weekend',
   'getting',
   'close',
   'greg',
   'leaving',
   'for',
   'the',
   'summer',
   'really',
   "don't",
   'want',
   'him'],
  'negative'),
 (['hu

In [41]:
# Classifier

# the list of words features need to be extracted from the tweets. 
# it is a list with every distinct words ordered by frequency of appearance

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_feature(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.most_common()
    # split the list of tuples to keep the words
    word_features, freq = map(list,zip(*word_features))
    return word_features

word_features = get_word_feature(get_words_in_tweets(tweets))
print(word_features[:50])



['the', 'and', 'you', 'for', 'have', 'that', 'but', "i'm", 'just', 'with', 'not', 'was', 'this', 'now', 'good', 'day', 'get', 'all', 'out', 'like', 'are', 'today', "it's", 'too', 'your', 'love', 'going', 'work', 'got', 'lol', 'time', 'back', 'from', 'what', 'one', 'will', 'know', 'about', 'really', "don't", 'had', 'can', 'see', 'some', "can't", 'its', 'still', 'night', 'well', 'new']


In [42]:
# feature extractor
# dictionnary indicating what words are contained in the input passed
def extract_features(document):
    document_words = set(document)
    features = {}
    count = 0
    for word in word_features:
        # cerate our dictionnary of 3000 words
        if count == 3000:
            break
        
        features['contain(%s)' %word] = (word in document_words)
        count = count + 1
    return features

In [43]:
len(word_features)

385038

In [44]:
# apply feature to our classifier 
training_set = nltk.classify.apply_features(extract_features, training_set)
test_set = nltk.classify.apply_features(extract_features, test_set)
print('done')

done


In [45]:
print(test_set)

[({'contain(miley)': False, 'contain(kevin)': False, 'contain(level)': False, 'contain(managed)': False, 'contain(bro)': False, 'contain(itunes)': False, 'contain(retweet)': False, 'contain(bummed)': False, 'contain(simple)': False, 'contain(catching)': False, 'contain(awkward)': False, 'contain(notice)': False, 'contain(warped)': False, 'contain(laughed)': False, 'contain(ways)': False, 'contain(manchester)': False, 'contain(interesting)': False, 'contain(copy)': False, 'contain(suddenly)': False, 'contain(stomach)': False, 'contain(exercise)': False, 'contain(reason)': False, 'contain(popcorn)': False, 'contain(editing)': False, 'contain(spirit)': False, 'contain(for)': False, 'contain(were)': False, 'contain(meet)': False, 'contain(youu)': False, 'contain(blessed)': False, 'contain(aswell)': False, 'contain(could)': False, 'contain(18th)': False, 'contain(lake)': False, 'contain(ring)': False, 'contain(pack)': False, 'contain(screwed)': False, "contain(you're)": False, 'contain(usua

In [46]:
# train our classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)
print('done')

done


In [47]:
print(nltk.classify.accuracy(classifier, test_set))

0.734


In [48]:
print(test_set)

[({'contain(miley)': False, 'contain(kevin)': False, 'contain(level)': False, 'contain(managed)': False, 'contain(bro)': False, 'contain(itunes)': False, 'contain(retweet)': False, 'contain(bummed)': False, 'contain(simple)': False, 'contain(catching)': False, 'contain(awkward)': False, 'contain(notice)': False, 'contain(warped)': False, 'contain(laughed)': False, 'contain(ways)': False, 'contain(manchester)': False, 'contain(interesting)': False, 'contain(copy)': False, 'contain(suddenly)': False, 'contain(stomach)': False, 'contain(exercise)': False, 'contain(reason)': False, 'contain(popcorn)': False, 'contain(editing)': False, 'contain(spirit)': False, 'contain(for)': False, 'contain(were)': False, 'contain(meet)': False, 'contain(youu)': False, 'contain(blessed)': False, 'contain(aswell)': False, 'contain(could)': False, 'contain(18th)': False, 'contain(lake)': False, 'contain(ring)': False, 'contain(pack)': False, 'contain(screwed)': False, "contain(you're)": False, 'contain(usua

In [49]:
# saracasm dataset header
columns = ['index', 'tweets']
# read the dataframe
saracasm_df = pd.read_csv('sarcasticTweets.csv', names = columns, error_bad_lines=False)
# get rid of NaN value
saracasm_df = saracasm_df.dropna(how='any') 
print(saracasm_df.shape[0])
saracasm_df

802


,index,tweets
1,0.0,Very low entries in this giveaway! Hop over an...
2,1.0,Summers over and it was shit. Only sunny day w...
3,2.0,Did you know that one of Bolshoi Ballet member...
4,3.0,I just love missing the bus! ☺ #sarcasm\n
5,4.0,If you're going into highschool and cant do at...
6,5.0,@babee_kayyy lol itold you iwasnt coming . :) ...
7,6.0,TThis 8:20 Friday class is gonna be grrreat!! ...
8,7.0,Gee wiz what a great day so far. You can cut t...
9,8.0,"Feel like crap, tomorrow should be great. #sar..."
10,9.0,If I could put into words how much I love waki...


In [50]:
# sarcastic tester
for tweet in saracasm_df.tweets:
    print(tweet)
#sarcastic_set = nltk.classify.apply_features(extract_features, sarcastic_tweet)

Very low entries in this giveaway! Hop over and check it out! http://t.co/OrUSN9ne

Summers over and it was shit. Only sunny day was a bbq in north west with @Kade_96 and @Jess_birch95 hahhahaaa #amazing #sarcasm

Did you know that one of Bolshoi Ballet members was born in Maranhão? Bruna Gaglianone is part of the Moscow group! http://t.co/LG3SME1l

I just love missing the bus! ☺ #sarcasm

If you're going into highschool and cant do at least 4 pirouettes you obviously go to a fantastic studio #sarcasm

@babee_kayyy lol itold you iwasnt coming . :) what we do in class ?

TThis 8:20 Friday class is gonna be grrreat!! #sarcasm

Gee wiz what a great day so far. You can cut the tension with a knife and they think im faking #great. #sarcasm

Feel like crap, tomorrow should be great. #sarcasm #throathurts #allergies

If I could put into words how much I love waking up at 5 am on Tuesday's I would.... 😒 #sarcasm.

I actually did something with my hair today #firsttimeinmonths #weird d

Thanks 

In [51]:
# tokenize the tweets
tweets_sarcastic = []
for tweet_s in saracasm_df['tweets']:
    tweet_s = tknzr.tokenize(tweet_s)
    words_filtered = []
    for e in tweet_s:
        e = e.lower()
        if ':' in e:
            e = e.replace(':', '')
        if '#' in e:
            e = e.replace('#', '')
        if '!' in e:
            e = e.replace('!', '')
        if '?' in e:
            e = e.replace('?', '')
        if ';' in e:
            e = e.replace(';', '')
        if '.' in e:
            e = e.replace('.', '')
        if ',' in e:
            e = e.replace(',', '')
        if '@' in e:
            e = e.replace('@', '')
        if len(e) >= 3:
            words_filtered.append(e)
    tweets_sarcastic.append(words_filtered)
print(tweets_sarcastic)

[['very', 'low', 'entries', 'this', 'giveaway', 'hop', 'over', 'and', 'check', 'out', 'http//tco/orusn9ne'], ['summers', 'over', 'and', 'was', 'shit', 'only', 'sunny', 'day', 'was', 'bbq', 'north', 'west', 'with', 'and', 'hahhahaaa', 'amazing', 'sarcasm'], ['did', 'you', 'know', 'that', 'one', 'bolshoi', 'ballet', 'members', 'was', 'born', 'maranhão', 'bruna', 'gaglianone', 'part', 'the', 'moscow', 'group', 'http//tco/lg3sme1l'], ['just', 'love', 'missing', 'the', 'bus', 'sarcasm'], ["you're", 'going', 'into', 'highschool', 'and', 'cant', 'least', 'pirouettes', 'you', 'obviously', 'fantastic', 'studio', 'sarcasm'], ['lol', 'itold', 'you', 'iwasnt', 'coming', 'what', 'class'], ['tthis', 'friday', 'class', 'gonna', 'grrreat', 'sarcasm'], ['gee', 'wiz', 'what', 'great', 'day', 'far', 'you', 'can', 'cut', 'the', 'tension', 'with', 'knife', 'and', 'they', 'think', 'faking', 'great', 'sarcasm'], ['feel', 'like', 'crap', 'tomorrow', 'should', 'great', 'sarcasm', 'throathurts', 'allergies'], [

In [52]:
# sarcastic sentiment tester
index = 0
for original_tweet in saracasm_df.tweets:
    # original tweet
    print(original_tweet)
    # tokenized tweet
    print(tweets_sarcastic[index])
    print(' ')
    # prediction
    print(classifier.classify(extract_features(tweets_sarcastic[index])))
    index = index + 1
    print(' ')
#test_set = nltk.classify.apply_features(extract_features, test_set)

Very low entries in this giveaway! Hop over and check it out! http://t.co/OrUSN9ne

['very', 'low', 'entries', 'this', 'giveaway', 'hop', 'over', 'and', 'check', 'out', 'http//tco/orusn9ne']
 
negative
 
Summers over and it was shit. Only sunny day was a bbq in north west with @Kade_96 and @Jess_birch95 hahhahaaa #amazing #sarcasm

['summers', 'over', 'and', 'was', 'shit', 'only', 'sunny', 'day', 'was', 'bbq', 'north', 'west', 'with', 'and', 'hahhahaaa', 'amazing', 'sarcasm']
 
positive
 
Did you know that one of Bolshoi Ballet members was born in Maranhão? Bruna Gaglianone is part of the Moscow group! http://t.co/LG3SME1l

['did', 'you', 'know', 'that', 'one', 'bolshoi', 'ballet', 'members', 'was', 'born', 'maranhão', 'bruna', 'gaglianone', 'part', 'the', 'moscow', 'group', 'http//tco/lg3sme1l']
 
positive
 
I just love missing the bus! ☺ #sarcasm

['just', 'love', 'missing', 'the', 'bus', 'sarcasm']
 
negative
 
If you're going into highschool and cant do at least 4 pirouettes you ob

positive
 
Well, today has started of to a WONDERFUL start. #Sarcasm

['well', 'today', 'has', 'started', 'wonderful', 'start', 'sarcasm']
 
negative
 
#sarcasm....learn it

['sarcasm', 'learn']
 
positive
 
What to do? Help please @BlackBerryHelp @BlackBerryPH http://t.co/8uX92LQU

['what', 'help', 'please', 'http//tco/8ux92lqu']
 
positive
 
@JonnyBones stay humble, but u turning down that fight is your choice no one else's it's not like you have the title oranything haha#sarcasm

['stay', 'humble', 'but', 'turning', 'down', 'that', 'fight', 'your', 'choice', 'one', "else's", "it's", 'not', 'like', 'you', 'have', 'the', 'title', 'oranything', 'haha', 'sarcasm']
 
negative
 
I think #ReachForTheStars is the next single from @iamwill featuring @britneyspears .....just look britney's last tweet

['think', 'reachforthestars', 'the', 'next', 'single', 'from', 'featuring', 'just', 'look', "britney's", 'last', 'tweet']
 
positive
 
CINCINNATI     (AP) -- Lunar pioneers plan to attend a priv

Today is a fucking lovely day! Nothing can go wrong!

['today', 'fucking', 'lovely', 'day', 'nothing', 'can', 'wrong']
 
negative
 
I must go like a page for me to look like ise a batty lord, just for one girl idk to get hair? vell chap my leg off and call me shorty

['must', 'like', 'page', 'for', 'look', 'like', 'ise', 'batty', 'lord', 'just', 'for', 'one', 'girl', 'idk', 'get', 'hair', 'vell', 'chap', 'leg', 'off', 'and', 'call', 'shorty']
 
positive
 
i chose brown sugar mini wheats

['chose', 'brown', 'sugar', 'mini', 'wheats']
 
positive
 
That plan to go to bed earlier in preparation for school went well. #sarcasm

['that', 'plan', 'bed', 'earlier', 'preparation', 'for', 'school', 'went', 'well', 'sarcasm']
 
positive
 
morning people. roadtrip today. unfortunately that means sitting in the car with my family for 3 or 4 hours. yay! #sarcasm

['morning', 'people', 'roadtrip', 'today', 'unfortunately', 'that', 'means', 'sitting', 'the', 'car', 'with', 'family', 'for', 'hours', 'ya

['chill', 'night', 'good', 'laughs', 'very', 'much', 'needed', 'breaking', 'night', 'watching', 'movie', 'loves', 'goodnight', 'tweethearts']
 
positive
 
Bring it on exams!!! #onmondaypa:) hehe

['bring', 'exams', 'onmondaypa', 'hehe']
 
positive
 
I get distracted way to easily

['get', 'distracted', 'way', 'easily']
 
negative
 
Is it weird that "Over My Dead Body" used to make me cry? Juss the chorus thoo.. Haha

['weird', 'that', 'over', 'dead', 'body', 'used', 'make', 'cry', 'juss', 'the', 'chorus', 'thoo', 'haha']
 
negative
 
Yes! I love waking up to @aferree9 and her little brother fighting! #sarcasm

['yes', 'love', 'waking', 'and', 'her', 'little', 'brother', 'fighting', 'sarcasm']
 
negative
 
@virginmedia are good! They got my house phone working for 24hrs! 😠 #sarcasm

['are', 'good', 'they', 'got', 'house', 'phone', 'working', 'for', '24hrs', 'sarcasm']
 
negative
 
@JHLFC @MartinSamuel_ i'dbe happy with Sturridge, need a finisher. Or maybe Jamie Hopkins,lol.

["i'dbe", '


['love', 'getting', 'home', 'from', 'work', 'knowing', 'that', 'less', 'than', '8hours', "you're", 'getting', 'back', 'there', 'again', 'sarcasm']
 
positive
 
I get warm fuzzies from 0-days I can't patch because of apps that NEED old versions of Java. Thanks Sunguard and ADP! #sarcasm

['get', 'warm', 'fuzzies', 'from', 'days', "can't", 'patch', 'because', 'apps', 'that', 'need', 'old', 'versions', 'java', 'thanks', 'sunguard', 'and', 'adp', 'sarcasm']
 
negative
 
Joseline needs to be restricted!

['joseline', 'needs', 'restricted']
 
positive
 
Woooo! When you're up to 12:30 finishing you're English paper! #sarcasm

['wooo', 'when', "you're", '1230', 'finishing', "you're", 'english', 'paper', 'sarcasm']
 
positive
 
Oh dang, I'm SOO happy you're SOO in love now.. ya know, considering that a week ago you weren't together. #sarcasm

['dang', "i'm", 'soo', 'happy', "you're", 'soo', 'love', 'now', 'know', 'considering', 'that', 'week', 'ago', 'you', "weren't", 'together', 'sarcasm']
 


 
@egerv24 You guys must be the reason construction gets done soooo fast ! #sarcasm #lazy

['you', 'guys', 'must', 'the', 'reason', 'construction', 'gets', 'done', 'sooo', 'fast', 'sarcasm', 'lazy']
 
positive
 
From Google &gt;&gt;&gt; Making it easier to cast your ballot http://t.co/C8esSUFZ

['from', 'google', 'making', 'easier', 'cast', 'your', 'ballot', 'http//tco/c8essufz']
 
negative
 
I'm still standing yesterday today and forever. That's why we praise Him. #sundaylineupmakessense

["i'm", 'still', 'standing', 'yesterday', 'today', 'and', 'forever', "that's", 'why', 'praise', 'him', 'sundaylineupmakessense']
 
negative
 
So glad I'm retaking physics. It's even more fun the second time around! Asian Accents are so easy to understand! #sarcasm

['glad', "i'm", 'retaking', 'physics', "it's", 'even', 'more', 'fun', 'the', 'second', 'time', 'around', 'asian', 'accents', 'are', 'easy', 'understand', 'sarcasm']
 
negative
 
Working on my day off! I love my job! #Sarcasm

['working', '

 
I always have to contain my grin of absolute joy when my boss tells me I can finish now. Yes, yes I can ;)

['always', 'have', 'contain', 'grin', 'absolute', 'joy', 'when', 'boss', 'tells', 'can', 'finish', 'now', 'yes', 'yes', 'can']
 
positive
 
Man, the #SEC must be crazy deep if Vandy is playing South Carolina this close. #sarcasm #eyeroll

['man', 'the', 'sec', 'must', 'crazy', 'deep', 'vandy', 'playing', 'south', 'carolina', 'this', 'close', 'sarcasm', 'eyeroll']
 
positive
 
this is going to be a fun semester... #sarcasm

['this', 'going', 'fun', 'semester', 'sarcasm']
 
positive
 
So excited to get up at 5 a.m. tomorrow!! #sarcasm ughh is it thursday yet????

['excited', 'get', 'tomorrow', 'sarcasm', 'ughh', 'thursday', 'yet']
 
negative
 
@SavRoell I am waiting for you....I want to do my home work #sarcasm

['waiting', 'for', 'you', 'want', 'home', 'work', 'sarcasm']
 
negative
 
@Daniel_Marnie pffft heskey and Carroll what a strike force! #sarcasm btw loved seeing you the o

negative
 
West Ham with Carlton Cole and Carroll up front. Going to be some free flowing football this season then #sarcasm

['west', 'ham', 'with', 'carlton', 'cole', 'and', 'carroll', 'front', 'going', 'some', 'free', 'flowing', 'football', 'this', 'season', 'then', 'sarcasm']
 
positive
 
"I'm sweating like a pedophile in a playground" -new favorite phrase of Luis. You can't imagine how thrilled I am by this #sarcasm

["i'm", 'sweating', 'like', 'pedophile', 'playground', 'new', 'favorite', 'phrase', 'luis', 'you', "can't", 'imagine', 'how', 'thrilled', 'this', 'sarcasm']
 
positive
 
Texas road house. Where winners are born. #shitlukesays. #sarcasm

['texas', 'road', 'house', 'where', 'winners', 'are', 'born', 'shitlukesays', 'sarcasm']
 
negative
 
your great @momabbeygal im so very happy you showed me 2 succeed on ur program i have made $3259 in 4 weeks @IM__NIA

['your', 'great', 'very', 'happy', 'you', 'showed', 'succeed', 'program', 'have', 'made', '3259', 'weeks']
 
positive

In [53]:
# saracasm detector 
# first step select two datasets of each 802 sarcastic and non sarcastic tweets
# our hypothesis is that there are no sarcastics tweets in the "normal" dataset

# store positive and negative tweets
tweet_negative = []
tweet_positive = []

# separate the negative and positive tweets
tweet_negative = tweets_df.loc[tweets_df['Sentiment'] == 0]
tweet_positive = tweets_df.loc[tweets_df['Sentiment'] == 1]

# negative label to saracasm
tweet_negative['Sentiment'].replace(0, 'negative',inplace=True)
tweet_positive['Sentiment'].replace(1, 'negative',inplace=True)

# only keep tweets and their label as a list
df_negative = list(zip(tweet_negative['SentimentText'], tweet_negative['Sentiment']))
df_positive = list(zip(tweet_positive['SentimentText'], tweet_positive['Sentiment']))

# non sarcastic dataframe
non_sarcastic_df = df_negative[:401] + df_positive[:401]

positive = []
for i in range(802):
    positive.append('positive')

# sarcastic dataframe
sarcastic_df = list(zip(saracasm_df.tweets, positive))

print(sarcastic_df)

/home/nicolas/.local/lib/python3.5/site-packages/pandas/core/generic.py:3813: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


[('Very low entries in this giveaway! Hop over and check it out! http://t.co/OrUSN9ne\n', 'positive'), ('Summers over and it was shit. Only sunny day was a bbq in north west with @Kade_96 and @Jess_birch95 hahhahaaa #amazing #sarcasm\n', 'positive'), ('Did you know that one of Bolshoi Ballet members was born in Maranhão? Bruna Gaglianone is part of the Moscow group! http://t.co/LG3SME1l\n', 'positive'), ('I just love missing the bus! ☺ #sarcasm\n', 'positive'), ("If you're going into highschool and cant do at least 4 pirouettes you obviously go to a fantastic studio #sarcasm\n", 'positive'), ('@babee_kayyy lol itold you iwasnt coming . :) what we do in class ?\n', 'positive'), ('TThis 8:20 Friday class is gonna be grrreat!! #sarcasm\n', 'positive'), ('Gee wiz what a great day so far. You can cut the tension with a knife and they think im faking #great. #sarcasm\n', 'positive'), ('Feel like crap, tomorrow should be great. #sarcasm #throathurts #allergies\n', 'positive'), ("If I could pu

In [54]:
# we create a global dataset with the tokenized tweets
# tokenize the tweets

tweets = []
for (words, sentiment) in sarcastic_df + non_sarcastic_df:
    words = tknzr.tokenize(words)
    words_filtered = []
    for e in words:
        e = e.lower()
        if ':' in e:
            e = e.replace(':', '')
        if '#' in e:
            e = e.replace('#', '')
        if '!' in e:
            e = e.replace('!', '')
        if '?' in e:
            e = e.replace('?', '')
        if ';' in e:
            e = e.replace(';', '')
        if '.' in e:
            e = e.replace('.', '')
        if ',' in e:
            e = e.replace(',', '')
        if '@' in e:
            e = e.replace('@', '')
        if len(e) >= 3:
            words_filtered.append(e)
    tweets.append((words_filtered, sentiment))
        
        
    #words_filtered = [e.lower() for e in words.split() if len(e) >= 3 and ':' not in e and '@' not in e and '!' not in e  and not '?' in e and not '.' in e and not ';' in e and not ',']
    #tweets.append((words_filtered, sentiment))
    
# shuffle the tweets dataset
random.shuffle(tweets)

In [55]:
# training and test set
# we keep 90% for the training set and 10% for the test set
limit_set = int(len(tweets)/10)
test_set = tweets[:limit_set]
training_set = tweets[limit_set:]
print(len(training_set))

1444


In [56]:

# testing the predictor
good_prediction = 0
wrong_prediction = 0
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0
total_tweet = 0
for tweet, sentiment in test_set:
    total_tweet = total_tweet + 1
    count = 0
    words = []
    for word in tweet:
        words.append(word)
        if count == int(len(tweet)/2): 
            first = classifier.classify(extract_features(words))
            words = []
        if count == int(len(tweet)-1):
            second = classifier.classify(extract_features(words))
            words = []
        count = count + 1
    print(tweet)
    print(first != second)
    if first != second:
        if sentiment == 'positive':
            true_positive = true_positive + 1
            good_prediction = good_prediction + 1
        else:
            false_positive = false_positive + 1
            wrong_prediction = wrong_prediction + 1
    else:
        if sentiment == 'positive':
            false_negative = false_negative + 1
        else:
            true_negative = true_negative + 1
    print("sarcasm", sentiment == 'positive')
        
print("accuracy = ", (true_positive + true_negative)/(true_positive + true_negative + false_positive + false_negative))
print("precision = ", true_positive / (true_positive + false_positive))
print("recall = ", true_positive / (true_positive + false_negative))
    
        
        
    
    



#test_set[0]

['guess', 'was', 'right', 'falcao', 'comin', 'lfc', 'sarcasm']
True
sarcasm True
['lmao']
False
sarcasm False
['imma', 'stop', 'snapping', 'tho']
False
sarcasm True
['well', 'that', 'sure', 'minimizes', 'the', 'impact', 'lol']
False
sarcasm False
[]
False
sarcasm False
['you', 'tease', 'wanna', 'hear', 'the', 'tracks']
False
sarcasm False
["that's", 'awesome', 'sarcasm']
False
sarcasm True
['unsure', 'annoyed', 'http//plurkcom/p/1230lz']
True
sarcasm False
['come', 'boro', 'but', 'who', 'the', 'hell', 'carayol']
False
sarcasm True
['nothing', 'pisses', 'off', 'more', 'then', 'fake', 'people', 'you', "don't", 'like', 'someone', "don't", 'act', 'like', 'you', 'areyouserious', 'likereally', 'sarcasm']
True
sarcasm True
['after', 'minutes', 'deliberation', 'has', 'been', 'decided', 'idaho', 'the', 'worst', 'state']
True
sarcasm True
['stay', 'til', 'talk', 'you', 'for', 'seconds', 'how', 'sweet', 'you', 'sarcasm', 'asshole', 'whatev']
True
sarcasm True
['pffft', 'heskey', 'and', 'carroll',

sarcasm False
['are', 'perfectly', 'schedule', 'nobody', 'waisting', 'time', 'today', "don't", 'mention', 'the', 'sun', 'again', 'dedicated', 'audience']
True
sarcasm False
['know', 'sold', 'out', 'though']
True
sarcasm False
['getting', 'hit', 'with', 'that', 'ntb', 'sarcasm']
False
sarcasm True
['how', 'plan', 'successful', 'sales', 'conference', 'meeting', 'http//tco/vtyamass']
False
sarcasm True
['awwwh', 'wanted', 'aiden', 'davis', 'win', 'him', 'bummed', 'didnt']
False
sarcasm False
['aww', "i'm", 'sorry', 'bae', 'kno', 'grindin', 'but', 'got', 'swear', 'needa', 'buy', 'artbook', 'that', 'construction', 'paper']
True
sarcasm False
['wait', 'what', "it's", 'cat', 'now', 'what', 'happening']
True
sarcasm True
['everyone', 'excited', 'about', 'the', 'gop', 'and', 'democrat', 'conventions', 'sarcasm', 'serioussarcasm', 'deepyawningmawofsarcaam']
False
sarcasm True
['cars', 'being', 'inspected', 'today', 'gonna', 'cost', 'over', '600', 'dollars']
False
sarcasm False
['not', 'feeling',